# Download Overture Maps with DuckDB 

## What is DuckDB?
DuckDB is designed to support analytical query workloads, also known as Online analytical processing (OLAP). It includes a columnar-vectorized query execution engine. This is more performant than traditional systems such as PostgreSQL, MySQL, or SQLite, which process each row sequentially. There are many plugins available. You can easily transfer your data in environments such as Amazon S3, Google Cloud Storage, postgresql using plugins. You can perform spatial analysis by installing the Spatial plugin.

### Creating a database

In [1]:
import duckdb
import overturemaps
import geopandas as gpd
db = duckdb.connect("data.db")

### Installing plug-ins for data access and spatial analysis
We install the "spatial" plugin to perform spatial analysis.
We are installing the "httpfs" plugin to access POI data in Amazon S3. Then we define the region as "us-west-2".

In [2]:
db.sql("""
INSTALL spatial;
INSTALL httpfs;
LOAD spatial;
LOAD httpfs;
SET s3_region='us-west-2';
""")


In [3]:
# Parameters
release = "2024-04-16-beta.0"
theme = "transportation"
ptype = "segment"
bbFile = "C:\\Data\\GitHub\\jetgeo\\OM2UML\\Data\\hamar.geojson"

Get the bounding box to search within. Use https://geojson.io/ to create a bounding box for your area of interest

In [6]:
# Bounding box
import geojson
from shapely.geometry import shape

def get_bbox(geometry):
    polygon = shape(geometry)
    return polygon.bounds

with open(bbFile) as f:
    gj = geojson.load(f)
    features = gj['features'][0]  # Assuming you want the first feature

bbox = get_bbox(features['geometry'])
print("Bounding Box (minx, miny, maxx, maxy):", bbox)


Bounding Box (minx, miny, maxx, maxy): (11.055774, 60.789123, 11.103776, 60.80348)


Get data and count the number of items

In [7]:
#bbox = -74.02169, 40.696423, -73.891338, 40.831263
table = overturemaps.record_batch_reader("building", bbox).read_all()

In [ ]:
# db.sql("""
#   create table places as 
#   select * from read_parquet('s3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=places/type=*/*')
# """)
try:
    db.sql("""DROP VIEW """ + ptype)
except:
    print("No existing table " + ptype)    

strSql= """CREATE VIEW """ + ptype + """ AS 
       SELECT * FROM read_parquet('s3://overturemaps-us-west-2/release/""" + release + """/theme="""+ theme + """/type=""" + ptype + """/*', filename=true, hive_partitioning=1)
       WHERE 
              bbox.minx > """ + str(bbox[0])+ """ AND 
              bbox.miny > """ + str(bbox[1])+ """ AND
              bbox.maxx < """ + str(bbox[2])+ """ AND 
              bbox.maxy < """ + str(bbox[3])+ """ ;
       """
# print(strSql)
db.sql(strSql)
db.sql("""select count(*) as count from """ + ptype).show()

Print all column names in the data set 

In [ ]:
res = db.sql('SELECT * FROM ' + ptype)
# Get the column names
column_names = res.columns

# Print the list of column names
print("Column Names:")
for name in column_names:
    print(name)

Write to json

In [ ]:

# Convert the result to a Pandas DataFrame
df = res.df()
# Remove some columns before exporting in order to reduce the file size
columns_to_drop = ['geometry', 'bbox','filename','theme', 'type','sources','version']
df.drop(columns=columns_to_drop, inplace=True)

# Write the DataFrame to a nicely formatted JSON file
output_file = ptype + ".json"
df.to_json(output_file, orient="records", lines=True, indent=2)

print(f"Data written to {output_file} in nicely formatted JSON format.")

Convert geometry and write to GeoJSON with only a few attributes

In [ ]:
db.sql("""COPY (
    SELECT ST_GeomFromWKB(""" + ptype + """.geometry) as geometry, id, subtype FROM """ + ptype + """
) TO '""" + ptype + """.geojson'
WITH (FORMAT GDAL, DRIVER 'GeoJSON');""")

Next: How to flatten the RoadType.class attribute?

In [ ]:
#Flatten the RoadType.class attribute to Segment.class



The content below is from https://github.com/Youssef-Harby/OvertureMapsDownloader 

In [ ]:
db.sql("""
    select * from """ + ptype + """ limit 25
""").show()


You can find the diagram of the POI data [here](https://docs.overturemaps.org/reference/places/place/). There are columns in the data that we need to preprocess.

In [ ]:
db.sql("""
    select names, categories, confidence,brand,addresses from places limit 5
""").show()

For example, in order to find out which category it is in the categories column, we need to get the information from the data held in the "struct" type. You can review the document to learn about DuckDB data types.
For example, to extract which country you are located in the "Addresses" column:

In [ ]:
db.sql("""
    select replace(json_extract(CAST(addresses AS JSON), '$[0].country')::varchar,'"','') as country from places limit 5
""").show()


After creating a column called “country” to extract country short names, we add the extracted data.

In [ ]:
try:
       db.sql("""ALTER TABLE places ADD COLUMN country VARCHAR;
              update places set country = replace(json_extract(CAST(places.addresses AS JSON), '$[0].country')::varchar,'"','')

       """)
except duckdb.Error as e:
    print(e)
    pass

We run the following query to add the POI data in Turkey to a separate table and obtain the address, category, name, geometry information.

In [ ]:
db.sql("""
       create or replace table turkey_places as (
              select
                     replace(json_extract(places.addresses::json,'$[0].locality'),'"','')::varchar as locality,
                     replace(json_extract(places.addresses::json,'$[0].region'),'"','')::varchar as region,
                     replace(json_extract(places.addresses::json,'$[0].postcode'),'"','')::varchar as postcode,
                     replace(json_extract(places.addresses::json,'$[0].freeform'),'"','')::varchar as freeform,

                     categories.main as categories_main,

                     replace(json_extract(places.names::json,'$.common[0].value'),'"','')::varchar as names,
                     confidence,
                     bbox,
                     st_transform(st_point(st_y(st_geomfromwkb(geometry)),st_x(st_geomfromwkb(geometry))),'EPSG:4326','EPSG:3857') as geom


              from places 
                     where country ='TR' 
       )


""")

Created table:

In [ ]:
db.sql("""
select * from turkey_places limit 5
""").df()

As an example, I will examine the POI data in Istanbul. I created two tables to obtain the POI points located within 500 m of the designated park points.

In [ ]:
db.sql("""
    create or replace table park_ist as (
        select * from turkey_places where locality = 'İstanbul' and categories_main='park'   
    );

    create or replace table poi_ist as (
        select * from turkey_places where locality = 'İstanbul' and categories_main <> 'park'
    )

""")

Number of POIs in Istanbul:

In [ ]:
db.sql(
    """
select count(*) from poi_ist

"""
)
'''
count
  181959
'''

Number of POIs designated as Parks in Istanbul:

In [ ]:
db.sql(
    """
select count(*) from park_ist

""")

'''
count
  492
'''


To query the POI points within 500 m of the points included in the park category:

In [ ]:
df = db.sql("""
              select  poi_ist.region as poi_ist_region,poi_ist.freeform as poi_ist_freeform,poi_ist.categories_main as poi_ist_categori ,
              park_ist.categories_main as park_categori , park_ist.names as park_names, park_ist.freeform as park_ist_freeform,

              st_distance(poi_ist.geom,park_ist.geom) as dist,
              ST_AsText(poi_ist.geom) as geom,
              ST_AsText(park_ist.geom) as geom2

              from poi_ist, park_ist 

              where ST_DWithin(poi_ist.geom, park_ist.geom,500) 
       """).to_df()

gdf = gpd.GeoDataFrame(df,geometry= gpd.GeoSeries.from_wkt(df['geom']),crs="EPSG:3857")
gdf.to_file("poi.geojson",driver="GeoJSON")